# PEFT 库 QLoRA 实战 - ChatGLM3-6B

In [1]:
!pip list | grep -E "torch|transformers|accelerate|bitsandbytes|datasets|optimum"

accelerate                0.34.2
bitsandbytes              0.46.1
datasets                  4.0.0
optimum                   1.27.0
torch                     2.7.0
torchaudio                2.7.0
torchcodec                0.5
torchvision               0.22.0
transformers              4.41.1


In [2]:
!nvidia-smi

Thu Aug  7 17:41:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.144                Driver Version: 570.144        CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 6000                Off |   00000000:B3:00.0  On |                  Off |
| 33%   40C    P8             22W /  260W |     775MiB /  24576MiB |      7%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
print(torch.cuda.is_available())  # 应输出True
print(torch.version.cuda)  # 应与nvcc -V显示的版本一致（或兼容）

True
12.6


In [4]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [5]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
train_data_path = 'HasturOfficial/adgen'    # 训练数据路径
eval_data_path = None                     # 验证数据路径，如果没有则设置为None
seed = 8                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 4                             # LoRA秩
lora_alpha = 32                           # LoRA alpha值
lora_dropout = 0.05                       # LoRA Dropout率
resume_from_checkpoint = None             # 如果从checkpoint恢复训练，指定路径
prompt_text = ''                          # 所有数据前的指令文本
compute_dtype = 'fp32'                    # 计算数据类型（fp32, fp16, bf16）

# 数据准备
### 下载数据集
### 从 Hugging Face 加载 adgen 数据集，并tokenize，shuffle

In [6]:
from datasets import load_dataset

dataset = load_dataset(train_data_path)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 114599
    })
    validation: Dataset({
        features: ['content', 'summary'],
        num_rows: 1070
    })
})

In [8]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(dataset["train"], num_examples=3)

,content,summary
0,类型#裙*材质#蕾丝*风格#复古*图案#复古*图案#波点*图案#蕾丝*裙型#抹胸裙*裙长#连衣裙*裙袖型#灯笼袖*裙款式#收腰,可以把它看做抹胸式连衣裙，抹胸之外叠搭了微透的波点蕾丝元素，同时用透明波点蕾丝剪裁出复古的灯笼袖造型，虽然手臂肌肤依然<UNK>裸露，却比大胆妩媚的抹胸裙多了一抹犹抱琵琶半遮面的朦胧感。腰部做了收腰设计，强调修长的下身比例，显得既乖巧又高挑。
1,类型#裤*颜色#蓝色*颜色#绿色*风格#简约*风格#清新*图案#字母*图案#文字*裤款式#抽绳,纯正的绿色和蓝色给人活力十足的清新感，让炎热夏季多了一丝清凉的视觉体验。泳裤两侧的条形设计贴合宝宝腰身，起到拉长宝宝腿部比例的作用。简约的字母达到提升泳裤时尚度的效果，弹力收腰设计带有抽绳方便儿童穿脱的同时，确保孩子游泳时不易脱落。宽腿版型不勒宝宝大腿根部，走路跑步行动自如，没有拘束感。
2,类型#裙*风格#淑女*风格#清新*图案#碎花*裙下摆#荷叶边*裙长#连衣裙*裙领型#圆领,一件清新靓丽的碎花连衣裙，上身具有甜美少女气息，并且给人以凉爽舒适的感觉。这件荷叶边裙摆剪裁的连衣裙，带有俏丽之美吸睛亮眼。圆领的款式，展现颈部曲线，能够塑造出修长的颈部曲线，碎花点缀展现出俏皮可爱的淑女风范，两袖的喇叭袖口设计，使其富有典雅大气的美感。


## 使用 ChatGLM3-6b Tokenizer 处理数据

In [10]:
from transformers import AutoTokenizer

# revision='b098244' 版本对应的 ChatGLM3-6B 设置 use_reentrant=False
# 最新版本 use_reentrant 被设置为 True，会增加不必要的显存开销
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

In [11]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}

In [12]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)

In [13]:
show_random_elements(tokenized_dataset, num_examples=1)

,input_ids,labels
0,"[64790, 64792, 30910, 33467, 31010, 56778, 30998, 33692, 31010, 34198, 30998, 32799, 31010, 37785, 30998, 56778, 54625, 31010, 44724, 30910, 31815, 55099, 54576, 54662, 40892, 38911, 31155, 31815, 33730, 32745, 37785, 33548, 54530, 44724, 31123, 49498, 53198, 55084, 54882, 31123, 35515, 31711, 34574, 54530, 35642, 31155, 54772, 34746, 34198, 44724, 40711, 33481, 54661, 56960, 31841, 31123, 32755, 54624, 58709, 54547, 39728, 37505, 31123, 36347, 35405, 54574, 55101, 32237, 31155, 2]","[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 30910, 31815, 55099, 54576, 54662, 40892, 38911, 31155, 31815, 33730, 32745, 37785, 33548, 54530, 44724, 31123, 49498, 53198, 55084, 54882, 31123, 35515, 31711, 34574, 54530, 35642, 31155, 54772, 34746, 34198, 44724, 40711, 33481, 54661, 56960, 31841, 31123, 32755, 54624, 58709, 54547, 39728, 37505, 31123, 36347, 35405, 54574, 55101, 32237, 31155, 2]"


### 数据集处理：shuffle & flatten

In [14]:
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)

In [15]:
tokenized_dataset = tokenized_dataset.flatten_indices()

# 定义 DataCollatorForChatGLM 类 批量处理数据

In [16]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}

In [17]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

# 训练模型
### 加载 ChatGLM3-6B 量化模型

In [18]:
from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

### 加载模型

In [19]:
# revision='b098244' 版本对应的 ChatGLM3-6B 设置 use_reentrant=False
# 最新版本 use_reentrant 被设置为 True，会增加不必要的显存开销
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True,
                                  revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [20]:
# 获取当前模型占用的 GPU显存（差值为预留给 PyTorch 的显存）
memory_footprint_bytes = model.get_memory_footprint()
memory_footprint_mib = memory_footprint_bytes / (1024 ** 2)  # 转换为 MiB

print(f"{memory_footprint_mib:.2f}MiB")

3739.69MiB


## 预处理量化模型
### 预处理量化后的模型，使其可以支持低精度微调训练

In [21]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training

kbit_model = prepare_model_for_kbit_training(model)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


### 自定义模型新增 Adapter

In [22]:
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

In [23]:
target_modules

['query_key_value']

# LoRA 适配器配置

In [24]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [25]:
qlora_model = get_peft_model(kbit_model, lora_config)

In [26]:
qlora_model.print_trainable_parameters()

trainable params: 974,848 || all params: 3,389,286,400 || trainable%: 0.0287626327477076


# 训练超参数配置

In [27]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=f"../model/nianhua-jiang/{model_name_or_path}",
    per_device_train_batch_size=16,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=4,                      # 梯度累积步数
    # per_device_eval_batch_size=8,                     # 每个设备的评估批量大小
    learning_rate=1e-3,                                 # 学习率
    num_train_epochs=1,                                 # 训练轮数
    lr_scheduler_type="linear",                         # 学习率调度器类型
    warmup_ratio=0.1,                                   # 预热比例
    logging_steps=10,                                   # 日志记录步数
    save_strategy="steps",                              # 模型保存策略
    save_steps=100,                                     # 模型保存步数
    # evaluation_strategy="steps",                      # 评估策略
    # eval_steps=500,                                   # 评估步数
    optim="adamw_torch",                                # 优化器类型
    fp16=True,                                          # 是否使用混合精度训练
)

In [28]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


### 训练参数（用于演示）

# 开始训练

In [29]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,5.145200
20,4.862000
30,4.216600
40,3.898600
50,3.723000
60,3.605600
70,3.541300
80,3.507100
90,3.502500
100,3.420900


/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs

TrainOutput(global_step=1790, training_loss=3.1605718623326475, metrics={'train_runtime': 17789.3614, 'train_samples_per_second': 6.442, 'train_steps_per_second': 0.101, 'total_flos': 6.921361834296607e+17, 'train_loss': 3.1605718623326475, 'epoch': 0.9995811810693843})

In [30]:
trainer.model.save_pretrained(f"models/demo/{model_name_or_path}")